Data Ingest
======

The data is a key component of any machine learning algorithm.
The ingest of the data,
including the cleanup and labelling stages,
is often the most time consuming.
It requires taking the raw data,
in our case Molecular Dyanmics trajectories, and
computing or extracting the quantities which will be used
for the Machine Learning.

In this example we will be using the relative orientations
of neighbouring molecules for our classification.
So by the end of this workbook we will have
a collection of data containing;

- the relative orientations of up to 6 nearest neighbours,
- the known classification, and
- temperature 

The input files
--------------

The input data for this tutorial is in the folder `data`,
which contains a series of `.gsd` files.
This is the default file format of [Hoomd-blue](http://glotzerlab.engin.umich.edu/hoomd-blue/)
a molecular dynamics package designed to run on GPUs
from the Glotzer group at the University of Michigan.

To read these files we are going to use the [gsd](http://gsd.readthedocs.io/en/latest/) package
that the glotzer group provide.
To read files from other simulation packages [MDAnalysis](https://www.mdanalysis.org/)
is a python package that will read nearly any file type.
To read all the files that we want,
we are going to use the [pathlib](https://docs.python.org/3/library/pathlib.html) module 
from python's (>= 3.4) standard library.

In [52]:
from pathlib import Path

import gsd.hoomd

In [53]:
# Create a Path object pointing to the directory holding
# all the input files we will be using
directory = Path('data/')
# Search for all files in the above directory which end in .gsd
input_files = directory.glob('*.gsd')

# input files is a generator, essentially a list of files,
# so we can iterate through each individually
for fname in input_files:
    # This is a context manager. We open the file and assign
    # it to the variable traj.
    with gsd.hoomd.open(str(fname)) as trj:
        # This extracts the first (and only) frame of the trajectory
        snap = trj[0]

For more information on context managers in Python
Jeff Knupp has a [good explanation](https://jeffknupp.com/blog/2016/03/07/python-with-context-managers/).

### Collate Simulation Data

This code is enough to read all the input files,
so the next step is to actually do something useful
with each of the files.

Within each of the filenames I have included 
the parameters of the simulation.
To extract then we can write a function 
to extract these values and return them to us as a [namedtuple](https://docs.python.org/3.6/library/collections.html#collections.namedtuple)

In [54]:
from collections import namedtuple

sim_params = namedtuple('sim_params', ['temperature', 'pressure', 'moment_inertia', 'crystal'])

def get_sim_params(fname: Path):
    """Extract the simulation parameters from a filename string."""
    # This gives just the filename as a string without directory or extension
    fname = fname.stem
    params = fname.split('-')
    return sim_params(temperature=float(params[2][1:]),
                      pressure=float(params[3][1:]),
                      moment_inertia=float(params[4][1:]),
                      crystal=params[5])

In [55]:
# Check our function works
get_sim_params(fname)

sim_params(temperature=1.0, pressure=0.4, moment_inertia=1.0, crystal='pg')

### Labelling Data

This has given us sufficient information about the simulation,
we now need to work out the classification of each molecule.
The configurations that I have prepared consist of two phases,
the middle 2/3 in the $x$ and $y$ directions is crystalline
while the remainer of the simulation cell is liquid.
This is probably easiest to understand using a picture;

In [56]:
from sdanalysis.figures.configuration import plot
from bokeh.plotting import show, output_notebook
output_notebook()

show(plot(snap))

Loading BokehJS ...

We can define molecules that have an $x$ position in the range
$
-L_x/3 < x < L_x/3
$
and have a $y$ position in the range
$
-L_y/3 < y < L_y/3
$
as being crystalline, with the crystal structure taken from the filename. 
The remaining molecules can be classed as the generic liquid.
As before we can write a simple function
that takes a snapshot and the cyrstal structure,
returning the annotated classification of all molecules in the simulation.
At the interface of the two phases
the classification is not well defined,
so for the purposes of training and testing, 
I am going to remove molecules within a distance of 3.5,
a bit over 1 neighbour shell,
to the boundary.

In [90]:
import numpy as np

def classify_mols(snapshot, crystal, boundary_buffer=3.5):
    """Classify molecules as crystalline, amorphous or boundary."""
    # This is the method of extracting the positions from a gsd snapshot
    position = snapshot.particles.position
    # This gets the details of the box from the simulation
    box = snapshot.configuration.box
    
    # All axes have to be True, True == 1, use product for logical and operation
    is_crystal = np.product(np.abs(position) < box[:3]/3, axis=1).astype(bool)
    boundary = np.logical_and(np.product(np.abs(position) < box[:3]/3+boundary_buffer, axis=1),
                              np.product(np.abs(position) > box[:3]/3-boundary_buffer, axis=1))
    
    # Create classification array
    classification = np.full(snapshot.particles.N, 'liq', dtype='<U4')
    classification[is_crystal] = crystal
    classification[boundary] = None
    return classification

### Compute training features

The final step is to compute the nearest neighbours for each of the molecules.
Most of the work of this function is done
using a function I wrote, `relative_orientations`.
This function uses the [freud](http://glotzerlab.engin.umich.edu/freud/index.html) package
again from the Glotzer group to compute the neighbours efficiently.
Then computes the relative orientation of the neighbour orientations using quaternion maths.

It should be noted that this the relative orientations function requires the orientations
to be expressed as quaternions. 

The parameters to the relative_orientations function, `max_radius` and `max_neighbours`
are passed to the algorithm computing the neighbour lists.
`max_radius` defines the maximum distance to search for neighbours,
beyond this distance molecules are not considered neighbours.
The `max_neighbours` parameter defines the maximum number of neighbours to find.
Where there are more molecules within the `max_radius` 
only the closest `max_neighbours` are returned.

In [91]:
from sdanalysis.order import relative_orientations

def compute_orientations(snapshot):
    """Compute the orientation of 6 nearest neighbours from a gsd snapshot."""
    # I am assuming an orthorhombic simulation cell
    box = snapshot.configuration.box[:3]
    return relative_orientations(box=box,
                                 position=snapshot.particles.position,
                                 orientation=snapshot.particles.orientation,
                                 max_radius=3.5,
                                 max_neighbours=6)

In [92]:
# Check our function works 
compute_orientations(snap)

array([[ 2.98683143,  0.81211901,  0.93995148,  1.6854409 ,  2.45859551,
        -0.        ],
       [ 1.4161737 ,  1.84473348,  1.16268682,  2.61352539,  2.23717332,
         0.57989556],
       [ 1.24541533,  1.35196137,  0.88213974,  3.0313127 ,  1.97685397,
         0.1217194 ],
       ..., 
       [ 1.71086359,  0.87404901,  0.84894603,  2.4572134 ,  2.09033251,
        -0.        ],
       [ 2.88447547,  0.84065735,  3.04268169,  0.34483594,  1.70382023,
         1.60555899],
       [ 0.77353138,  2.88240266,  1.92576969,  0.28307882,  0.38000262,
         0.33549434]], dtype=float32)

Loading the Data
----------------

Now with all the parts in place we can load all the data into a pandas DataFrame.
Taking the code we had at the start to read in all the snapshots,
we can now apply the functions we have just written,

- `get_sim_params`,
- `classify_mols`, and
- `compute_orientations`

to process the data into a succinct and usable form.

In [93]:
import pandas

directory = Path('data/')
input_files = directory.glob('*.gsd')
all_dataframes = []

for fname in input_files:
    with gsd.hoomd.open(str(fname)) as trj:
        snap = trj[0]
        # Get simulation parameters
        params = get_sim_params(fname)
        # Classify all molecules
        classes = classify_mols(snap, params.crystal)
        # Compute the orientations
        orientations = compute_orientations(snap)
        
        # Create dataframe
        df = pandas.DataFrame({
            'temperature': params.temperature,
            'pressure': params.pressure,
            'crystal': params.crystal,
            'class': classes,
            'orient0': orientations[:, 0],
            'orient1': orientations[:, 1],
            'orient2': orientations[:, 2],
            'orient3': orientations[:, 3],
            'orient4': orientations[:, 4],
            'orient5': orientations[:, 5],
        })
        
        # Remove molecules close to interface
        df = df[df['class'] != 'None']
        
        all_dataframes.append(df)

# Collate list of dataframes into single large dataframe
training_dataset = pandas.concat(all_dataframes)

# Save dataset to file
training_dataset.to_hdf('data/training_data.h5', key='trimer')

In [94]:
# Check the dataframe contains reasonable data
training_dataset.head()

,class,crystal,orient0,orient1,orient2,orient3,orient4,orient5,pressure,temperature
0,liq,p2gg,1.363178,0.796889,0.953538,1.798195,2.573618,-0.00000,0.5,1.0
1,liq,p2gg,1.244835,0.285656,0.978083,2.072978,0.335787,-0.00000,0.5,1.0
2,liq,p2gg,2.215736,3.134794,0.600781,1.214479,1.712630,-0.00000,0.5,1.0
3,liq,p2gg,0.807430,2.384640,1.192246,0.183372,2.687490,-0.00000,0.5,1.0
4,liq,p2gg,0.648713,2.608390,3.129185,1.873710,2.221143,1.30696,0.5,1.0


At this point we have our dataset saved to a file
so we don't have to do this again,
and are ready to start the process 
of training some machine learning algorithms.

The next step in this tutorial is [finding a good model](02_Lets_Find_A_Model.ipynb)